In [19]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

df = pd.read_csv('EnglandLeagues2.csv')

In [20]:
# Data preprocessing
df = df[[c for c in df if c not in ['FTR']] + ['FTR']]
for i in range(0, len(df['FTR'])):
    if df['FTR'][i] == 'H':
        df.loc[i, 'FTRB'] = 1
    elif df['FTR'][i] == 'D':  # Account for draw outcome
        df.loc[i, 'FTRB'] = 0.5
    else:
        df.loc[i, 'FTRB'] = 0

df = df.drop(['FTR', 'Date', 'Div', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HTR', 'Time'], axis=1)
df.dropna(inplace=True)
# Splitting the dataset for training and validation
df_2 = df.iloc[3000:,:]
df = df.iloc[:3000,:]

In [21]:
# Feature encoding and more preprocessing
X = df.drop('FTRB', axis=1)
Y = df.drop(['HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA',
       'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH',
       'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5',
       'Avg>2.5', 'Avg<2.5'], axis=1)
mean = X.mean(axis=0)
X -= mean
std = X.std(axis=0)
X /= std
df[['HomeTeam', 'AwayTeam']] = df[['HomeTeam', 'AwayTeam']].apply(LabelEncoder().fit_transform)

C:\Users\ziadh\AppData\Local\Temp\ipykernel_3640\937144811.py:7: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  mean = X.mean(axis=0)


In [22]:
# Creating the model
model = Sequential()
model.add(Dense(8, input_dim = len(X.columns), activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
# Compiling and running fit on the model
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=['accuracy', 'precision', 'recall', 'f1'])
model.fit(x=X, y=Y, epochs=256, verbose=1)

In [49]:
prediction = model.predict(X)

117/117 [==============================] - 0s 545us/step


In [41]:
my_accuracy = accuracy_score(Y, prediction.round())
print(my_accuracy)

0.6632543103448276


In [ ]:
for i in range(0, len(df['B365H'])):
    if i in df['B365B'].index and df['B365B'][i] < 2:
        Z.loc[i, 'B365B'] = int(1)
    else:
        Z.loc[i, 'B365B'] = int(0)
for i in range(0, len(Z['Pred'])):
    if i in Z['Pred'].index and Z['Pred'][i] < 0.5:
        Z.loc[i, 'PredB'] = int(0)
    else:
        Z.loc[i, 'PredB'] = int(1)
for i in range(0, len(Z['PredB'])):
    if i in Z['Pred'].index and Z['PredB'][i] == Z['FTR'][i] and Z['B365B'][i] != Z['FTR'][i]:
        Z.loc[i, 'PredBeat'] = int(1)
    else:
        Z.loc[i, 'PredBeat'] = int(0)
for i in range(0, len(Z['Pred'])):
    if i in Z['Pred'].index and Z['PredB'][i] == Z['FTR'][i]:
        Z.loc[i, 'PredWin'] = int(1)
    else:
        Z.loc[i, 'PredWin'] = int(0)
Z['FTR'] = df['FTRB']
Z['Pred'] = prediction

In [ ]:
# Convert the float labels back to their original categorical form
y_true = df['FTRB'].map({1: 'H', 0.5: 'D', 0: 'A'}).values
y_pred = prediction_rounded.flatten().map({1: 'H', 0.5: 'D', 0: 'A'}).values

# Compute precision, recall, and F1-score for each class separately
precision = precision_score(y_true, y_pred, average=None, labels=['H', 'D', 'A'])
recall = recall_score(y_true, y_pred, average=None, labels=['H', 'D', 'A'])
f1 = f1_score(y_true, y_pred, average=None, labels=['H', 'D', 'A'])

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)